## Implementing search

We have to implement the search now, in order to do so we have to keep some things in mind:

- From each inverted index, there will be a row of doc_ids received, for each doc_id, we need to gather the document information we have to display
- We also need to employ a ranking algorithm, that can process both single word and multiple word queries
- 

In [1]:
from searching import search
from docdata.getdocdata.getjson import get_docdata

### We will start by checking if we can retrieve the docdata from its doc_id

In [12]:
%%time
get_docdata(21321)

CPU times: user 13 µs, sys: 3 µs, total: 16 µs
Wall time: 17.9 µs


{'url': 'https://www.thesun.co.uk/sport/18399667/tobin-heath-released-arsenal-contract/',
 'title': 'Tobin Heath’s Arsenal spell ends with USA ace released early from contract by mutual consent following injury',
 'date': '2022-04-28',
 'chars500': 'TOBIN HEATH ’ s Arsenal stint has come to an end with the forward leaving the club by mutual agreement following a hamstring injury .\nThe USA forward has been contending with a minor problem which has ruled her out of the Gunners ’ last three games of the season .\nHeath ’ s one-year contract was due to expire at the end of the 2021-22 term .\nHowever , hamstring issues have put the player , 33 , out of contention for Arsenal ’ s games against Aston Villa , Tottenham and West Ham .\nThe club are ta',
 'author': 'Sandra Brobbey'}

### Implemented this pretty well